In [1]:
import tensorlayer as tl
import os
import matplotlib
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import requests
from sklearn import preprocessing
from tensorlayer.prepro import imresize
from itertools import compress

BATCH_SIZE = 20
NUM_CLASSES = 200
NUM_IMAGES_PER_CLASS = 500
NUM_IMAGES = NUM_CLASSES * NUM_IMAGES_PER_CLASS
TRAINING_IMAGES_DIR = './tiny-imagenet-200/train/'
TRAIN_SIZE = NUM_IMAGES

NUM_VAL_IMAGES = 10000
VAL_IMAGES_DIR = './tiny-imagenet-200/val/'

def downsample_fn(x):
    x = imresize(x, size=[32,32], interp='bicubic', mode = None)
    x = x/(255./2.)
    x = x-1.
    return x

def load_tiny_imagenet(path = './tiny-imagenet-200/'):
    data_dict = dict()
    train_dir = path + 'train/'
    test_dir = path + 'val/'
    types = sorted(os.listdir(train_dir))
    train_images = list()
    train_label_name = list()
    for t in types: 
        print(t)
        tmp_path = './tiny-imagenet-200/train/'+t+'/images/'
        file_list = tl.files.load_file_list(path=tmp_path, regx = '.*.JPEG', printable=False)
        tmp=tl.vis.read_images(file_list, path = tmp_path, n_threads=32, printable=False)
#         filter out those images with less dim
        tmp = list(filter(lambda x: x.shape == (64, 64, 3), tmp))
        print(np.shape(tmp))
        
        train_images = train_images+tmp
        tmp_label = list(map(lambda x: x.split('_')[0], file_list))
        train_label_name = train_label_name+tmp_label[:len(tmp)]
#  deal with the test data
    test_file_list= list(map(lambda x: 'val_'+str(x)+'.JPEG', range(10000)))
    test_images = tl.vis.read_images(test_file_list, path = test_dir+'/images/', 
                                     n_threads=32, printable=False)
    test_label_name = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', 
                                  sep='\t', header=None)[1].values
    bool_list = list(map(lambda x: x.shape==(64,64,3),test_images))
    test_images = np.array(list(compress(test_images, bool_list)))
    test_label_name = test_label_name[bool_list]
    
    le = preprocessing.LabelEncoder()
    training_le = le.fit(train_label_name)
    train_label = training_le.transform(train_label_name)
    test_label = training_le.transform(test_label_name)
#     reshape images
    print('reshape images')
    train_images = tl.prepro.threading_data(np.array(train_images), fn=downsample_fn)
    test_images = tl.prepro.threading_data(np.array(test_images), fn=downsample_fn)
    
    data_dict['X_test'] = np.array(test_images).astype(np.float32)
    data_dict['X_train'] = np.array(train_images).astype(np.float32)
    data_dict['Y_train'] = train_label.astype(np.int32)
    data_dict['Y_test'] = test_label.astype(np.int32)
    data_dict['X_valid'] = np.zeros([0])
    data_dict['Y_valid'] = np.zeros([0])  
    return data_dict

In [2]:
a = load_tiny_imagenet()

n01443537
(500, 64, 64, 3)
n01629819
(500, 64, 64, 3)
n01641577
(500, 64, 64, 3)
n01644900
(498, 64, 64, 3)
n01698640
(496, 64, 64, 3)
n01742172
(499, 64, 64, 3)
n01768244
(492, 64, 64, 3)
n01770393
(497, 64, 64, 3)
n01774384
(497, 64, 64, 3)
n01774750
(495, 64, 64, 3)
n01784675
(500, 64, 64, 3)
n01855672
(498, 64, 64, 3)
n01882714
(499, 64, 64, 3)
n01910747
(499, 64, 64, 3)
n01917289
(499, 64, 64, 3)
n01944390
(495, 64, 64, 3)
n01945685
(497, 64, 64, 3)
n01950731
(500, 64, 64, 3)
n01983481
(500, 64, 64, 3)
n01984695
(500, 64, 64, 3)
n02002724
(499, 64, 64, 3)
n02056570
(500, 64, 64, 3)
n02058221
(498, 64, 64, 3)
n02074367
(498, 64, 64, 3)
n02085620
(497, 64, 64, 3)
n02094433
(497, 64, 64, 3)
n02099601
(494, 64, 64, 3)
n02099712
(495, 64, 64, 3)
n02106662
(492, 64, 64, 3)
n02113799
(491, 64, 64, 3)
n02123045
(492, 64, 64, 3)
n02123394
(492, 64, 64, 3)
n02124075
(492, 64, 64, 3)
n02125311
(497, 64, 64, 3)
n02129165
(493, 64, 64, 3)
n02132136
(500, 64, 64, 3)
n02165456
(500, 64, 64, 3)
n

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import combinations
from itertools import chain
from collections import namedtuple
import pickle
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
import os.path
import shutil
import inspect
import matplotlib.pyplot as plt
import json
from pprint import pprint
from copy import copy
from nn_lib import *
from train_utils import *
from cifar100_utils import *
from train_procedures import *
np.random.seed(1997)
ALGORITHM=sys.argv[1]
BASE_DIR=sys.argv[2]
if ALGORITHM=='full_data':
    CLASS_BATCH_SIZE=int(sys.argv[3])

In [4]:
data_dict=a

In [5]:
# for testing
ALGORITHM="SupportNet"

In [6]:
#non-incremental settings
if ALGORITHM == 'ft':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':200,
                  'base_dir':BASE_DIR,
                  'class_batch_size':4,
                  'use_theoretical_mean':True}
#icarl settings
elif ALGORITHM == 'icarl':
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'icarl_exemplars',
                  'train_method':'train_distillation',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':200,
                  'base_dir':BASE_DIR,
                  'class_batch_size':4,
                  'use_theoretical_mean':True}
#SupportNet settings
elif ALGORITHM == 'SupportNet':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':200,
                  'base_dir':BASE_DIR,
                  'class_batch_size':4,
                  'use_theoretical_mean':True}
#ewc settings
elif ALGORITHM == 'ewc':
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':200,
                  'base_dir':BASE_DIR,
                  'class_batch_size':4,
                  'use_theoretical_mean':True}
elif ALGORITHM == 'full_data':
    hyper_params={'beta':1e-5,
                  'initial_lr':2,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar100',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':200,
                  'base_dir':BASE_DIR,
                  'class_batch_size':CLASS_BATCH_SIZE,
                  'use_theoretical_mean':True}
else:
    assert False

In [7]:
tf_tensors,tf_variables,tf_networks=build_graph(hyper_params,fixed_params)

Use ResNet32


/home/liy0f/incremental_learning_ec/SupportNet/src_image_data/train_procedures.py:50: UserWarning: when use ewc, loss_function should be softmax
  warnings.warn('when use ewc, loss_function should be softmax')


Use ResNet32
Use ResNet32


In [8]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [10]:
if ALGORITHM=='full_data':
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=1,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)
else:
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=50,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)

{'base_dir': '/run/user/142792/jupyter/kernel-abf412da-0d16-493b-a929-f2cc3b5033b4.json',
 'class_batch_size': 4,
 'dataset': 'cifar100',
 'net_type': 'ResNet32',
 'random_seed': 1997,
 'total_num_classes': 100,
 'use_theoretical_mean': True}
{'beta': 1e-05,
 'ewc_lambda': 0.01,
 'exemplars_set_size': 2000,
 'final_train_epochs': 5,
 'initial_lr': 0.1,
 'loss_function': 'sigmoid_cross_entropy_with_logits',
 'lr_reduction_epoch': [49, 63],
 'lr_reduction_rate': 10,
 'optimizer': 'momentum',
 'primary_exemplars': 'svm_exemplars',
 'sample_weight': 0,
 'se': True,
 'shuffle_class_ord': False,
 'test_batch_size': 64,
 'train_batch_size': 64,
 'train_method': 'train_plain',
 'use_fixedsize_exemplars': True}
num_iterations exceeds maximum allowed number, reset to 25.0
Y_train class info:
Counter({0: 500, 1: 500, 2: 500, 3: 498})
Y_test class info:
Counter({0: 50, 1: 50, 2: 50, 3: 50})
===========Iteration 1=============
Using classes [0, 1, 2, 3]


/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Class batch pretrain evaluation (plain method)
	Train loss: 0.032392
	Train class loss: 0.014933
	Train reg loss: 0.017459
	Validation loss: 0.032091
	Validation class loss: 0.014631
	Validation reg loss: 0.017459
	Top1 train accuracy: 0.250250
	Top5 train accuracy: 0.250250
	Top1 validation accuracy: 0.250000
	Top5 validation accuracy: 0.250000
[[50  0  0  0]
 [50  0  0  0]
 [50  0  0  0]
 [50  0  0  0]]
Epoch 1
	Train loss: 0.317261
	Train class loss: 0.299895
	Train reg loss: 0.017366
	Validation loss: 0.018285
	Validation class loss: 0.000729
	Validation reg loss: 0.017555
	Top1 train accuracy: 0.219720
	Top5 train accuracy: 0.713714
	Top1 validation accuracy: 0.235000
	Top5 validation accuracy: 0.990000
[[ 0 47  0  3]
 [ 0 47  0  3]
 [ 0 48  0  2]
 [ 0 50  0  0]]
mbi=274364416
time= 4.979381
saving model parameters...
Epoch 2
	Train loss: 0.054996
	Train class loss: 0.037528
	Train reg loss: 0.017468
	Validation loss: 0.017921
	Validation class loss: 0.000308
	Validation reg loss:

Epoch 20
	Train loss: 0.032905
	Train class loss: 0.015488
	Train reg loss: 0.017417
	Validation loss: 0.017702
	Validation class loss: 0.000165
	Validation reg loss: 0.017536
	Top1 train accuracy: 0.621121
	Top5 train accuracy: 0.992993
	Top1 validation accuracy: 0.640000
	Top5 validation accuracy: 1.000000
[[45  0  4  1]
 [ 1 26 11 12]
 [ 2  6 30 12]
 [ 5  5 13 27]]
mbi=274364416
time= 2.935225
saving model parameters...
Epoch 21
	Train loss: 0.032653
	Train class loss: 0.015244
	Train reg loss: 0.017410
	Validation loss: 0.017697
	Validation class loss: 0.000168
	Validation reg loss: 0.017529
	Top1 train accuracy: 0.636637
	Top5 train accuracy: 0.992993
	Top1 validation accuracy: 0.600000
	Top5 validation accuracy: 1.000000
[[45  0  5  0]
 [ 0 23 18  9]
 [ 5  4 32  9]
 [ 5  4 21 20]]
mbi=274364416
time= 3.037447
Epoch 22
	Train loss: 0.032302
	Train class loss: 0.014900
	Train reg loss: 0.017402
	Validation loss: 0.017676
	Validation class loss: 0.000155
	Validation reg loss: 0.0175

Epoch 41
	Train loss: 0.028914
	Train class loss: 0.011669
	Train reg loss: 0.017245
	Validation loss: 0.017490
	Validation class loss: 0.000127
	Validation reg loss: 0.017362
	Top1 train accuracy: 0.731231
	Top5 train accuracy: 0.992492
	Top1 validation accuracy: 0.720000
	Top5 validation accuracy: 1.000000
[[47  0  3  0]
 [ 0 39  7  4]
 [ 2  2 37  9]
 [ 5  4 20 21]]
mbi=274364416
time= 2.244441
Epoch 42
	Train loss: 0.028710
	Train class loss: 0.011473
	Train reg loss: 0.017236
	Validation loss: 0.017477
	Validation class loss: 0.000123
	Validation reg loss: 0.017353
	Top1 train accuracy: 0.731231
	Top5 train accuracy: 0.992492
	Top1 validation accuracy: 0.720000
	Top5 validation accuracy: 1.000000
[[47  0  3  0]
 [ 0 40  5  5]
 [ 1  1 22 26]
 [ 3  5  7 35]]
mbi=274364416
time= 2.302634
Epoch 43
	Train loss: 0.028564
	Train class loss: 0.011336
	Train reg loss: 0.017228
	Validation loss: 0.017468
	Validation class loss: 0.000124
	Validation reg loss: 0.017345
	Top1 train accuracy: 0.

...

SVM evaluation
	Best top1 validation accuracy: 0.338462
	Best top1 cumul accuracy: 0.221298
	Best top1 ori accuracy: 0.360000
	Report string of cumul dataset
              precision    recall  f1-score   support

           0       0.50      0.60      0.55        50
           1       0.44      0.56      0.49        50
           2       0.17      0.18      0.17        50
           3       0.10      0.10      0.10        50
           4       0.23      0.29      0.25        49
           5       0.08      0.06      0.07        50
           6       0.19      0.20      0.20        50
           7       0.13      0.14      0.13        50
           8       0.32      0.50      0.39        50
           9       0.15      0.12      0.13        50
          10       0.08      0.08      0.08        50
          11       0.28      0.28      0.28        47
          12       0.33      0.34      0.33        50
          13       0.53      0.56      0.54        50
          14       0.43      0.

Theoretical mean evaluation
	Best top1 validation accuracy: 0.400000
	Best top5 validation accuracy: 0.625641
	Best top1 cumul accuracy: 0.283976
	Best top5 cumul accuracy: 0.568763
	Best top1 ori accuracy: 0.445000
	Best top5 ori accuracy: 0.635000
	Report string of cumul dataset
              precision    recall  f1-score   support

           0       0.56      0.72      0.63        50
           1       0.43      0.60      0.50        50
           2       0.19      0.26      0.22        50
           3       0.31      0.20      0.24        50
           4       0.30      0.37      0.33        49
           5       0.16      0.10      0.12        50
           6       0.14      0.18      0.16        50
           7       0.18      0.18      0.18        50
           8       0.34      0.56      0.42        50
           9       0.17      0.18      0.17        50
          10       0.14      0.14      0.14        50
          11       0.32      0.26      0.28        47
          12   

/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liy0f/.conda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Final evaluation on all classes (plain method)
	Top1 train accuracy: 0.128795
	Top5 train accuracy: 0.281842
	Top1 validation accuracy: 0.115541
	Top5 validation accuracy: 0.256611


IndexError: index 107 is out of bounds for axis 1 with size 100